### Based on Cascade Classifiers paper

### Dependencies
Face and Eye Models by OpenCV:
- https://github.com/opencv/opencv/blob/4.x/data/haarcascades/haarcascade_frontalface_default.xml
- https://github.com/opencv/opencv/blob/4.x/data/haarcascades/haarcascade_eye.xml

### Import Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image

### Get the Models

In [2]:
face_model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
emotion_model = load_model('facial_emotion_detection_model_50E.h5')
age_model = load_model('age_model_50E.h5')
gender_model = load_model('gender_model_50E.h5')

In [3]:
## Define Labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
gender_labels = ["Male", "Female"]

### Implemented Using Video Capture
Functions similarly when using image. Only differs in implementing video capture in OpenCV.

In [4]:
video = cv2.VideoCapture(0) #initialize video capturing

while True:
    ret, img = video.read()
    
    labels = []
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert image read to grayscale
    faces = face_model.detectMultiScale(img_gray, 1.3, 5) #allows face detection despite movement of face
    
    #x,y,w,h change for every movement of the face
    for (x,y,w,h) in faces:
        emotion_color = (0, 0, 0)
        gender_color = (0, 0, 0)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2) #draw a rectangle
        #Get Face ROI
        roi_gray = img_gray[y:(y+h), x:(x+w)] #Selects only the detected face
        roi_gray = cv2.resize(roi_gray, (48,48), interpolation=cv2.INTER_AREA) #emotion detection model has 48x48 training
        
        roi = roi_gray.astype('float') / 255.0 #normalize by scaling values to between 0 and 1
        roi = img_to_array(roi) #convert to array for processing
        roi = np.expand_dims(roi, axis=0) #expand dimensions for proper prediction (N,W,H,C)
        roi_color = img[y:y+h, x:x+w]
        roi_color = cv2.resize(roi_color, (48,48), interpolation=cv2.INTER_AREA)
        
        #Emotion Prediction
        emotion_predict = emotion_model.predict(roi)[0]
        emotion_label = emotion_labels[emotion_predict.argmax()]
        if emotion_label == 'Happy' or emotion_label == 'Surprise':
            emotion_color = (0, 255, 0)
        elif emotion_label == 'Neutral':
            emotion_color = (255, 0, 0)
        else:
            emotion_color = (0, 0, 255)
        emotion_label_pos = (x, y) #position of the emotion label relative to the bounding box
        cv2.putText(img, emotion_label, emotion_label_pos, cv2.FONT_HERSHEY_SIMPLEX, 1, emotion_color, 2)
        
        #Age Prediction
        age_predict = age_model.predict(np.array(roi_color).reshape(-1,48,48,3))
        age_label = round(age_predict[0,0])
        age_label_pos = (x,y-100)
        cv2.putText(img, "Age: " + str(age_label), age_label_pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        #Gender Prediction
        gender_predict = gender_model.predict(np.array(roi_color).reshape(-1,48,48,3))
        gender_predict = (gender_predict >= 0.5).astype(int)[:,0]
        gender_label = gender_labels[gender_predict[0]]
        if gender_label == "Male":
            gender_color = (255, 0, 0)
        elif gender_label == "Female":
            gender_color = (0, 0, 255)
        gender_label_pos = (x,y-50)
        cv2.putText(img, "Sex: " + str(gender_label), gender_label_pos, cv2.FONT_HERSHEY_SIMPLEX, 1, gender_color, 2)
        
    cv2.imshow('img', img)
    k = cv2.waitKey(30) & 0xFF
    if k == 27: #ESC to end
        break

video.release() #end video capturing
cv2.destroyAllWindows()